In [2]:
N = 5

In [3]:
adj = [[[] for _ in range(N)] for _ in range(N)]
for i in range(N):
    for q in range(N):
        for dy, dx in [
            (-1, 0),
            (1, 0),
            (0, 1),
            (0, -1)
        ]:
            ny, nx = i + dy, q + dx
            if nx >= 0 and nx < N and ny >= 0 and ny < N:
                adj[i][q].append((ny, nx))

In [51]:
# Given some starting position at the top, approximate the probability of landing at the corresponding bottom mask
def simStart():
    probs = [[0 for _ in range(N)] for _ in range(N)]
    probs[2][2] = 1
    prob_lower = [0 for _ in range(N)]
    ev_lower = [0 for _ in range(N)]
    for step in range(10_000):
        nxt = [[0 for _ in range(N)] for _ in range(N)]
        for i in range(N):
            for q in range(N):
                for y, x in adj[i][q]:
                    nxt[y][x] += probs[i][q] / len(adj[i][q])
        # Extract probs
        for i in range(N):
            prob_lower[i] += nxt[0][i]
            ev_lower[i] += nxt[0][i] * (step + 1)
            nxt[0][i] = 0
        probs = nxt.copy()
    return prob_lower, ev_lower

In [70]:
# Given some starting position at the top, approximate the probability of landing at the corresponding bottom mask
from functools import cache
@cache
def simTop(topPos, botMask):
    probs = [[0 for _ in range(N)] for _ in range(N)]
    probs[0][topPos] = 1
    prob_lower = [0 for _ in range(N)]
    ev_lower = [0 for _ in range(N)]
    for step in range(10_000):
        nxt = [[0 for _ in range(N)] for _ in range(N)]
        for i in range(N):
            for q in range(N):
                for y, x in adj[i][q]:
                    nxt[y][x] += probs[i][q] / len(adj[i][q])
        # Extract probs
        for i in range(N):
            if botMask[i]:
                prob_lower[i] += nxt[4][i]
                ev_lower[i] += nxt[4][i] * (step + 1)
                nxt[4][i] = 0
        probs = nxt.copy()
    return prob_lower, ev_lower

In [258]:
# Given some starting position at the top, approximate the probability of landing at the corresponding bottom mask
@cache
def simBot(botPos, topMask):
    probs = [[0 for _ in range(N)] for _ in range(N)]
    probs[4][botPos] = 1
    prob_lower = [0 for _ in range(N)]
    ev_lower = [0 for _ in range(N)]
    for step in range(10_000):
        nxt = [[0 for _ in range(N)] for _ in range(N)]
        for i in range(N):
            for q in range(N):
                for y, x in adj[i][q]:
                    nxt[y][x] += probs[i][q] / len(adj[i][q])
        # Extract probs
        for i in range(N):
            if topMask[i]:
                prob_lower[i] += nxt[0][i]
                ev_lower[i] += nxt[0][i] * (step + 1)
                nxt[0][i] = 0
        probs = nxt.copy()
    return prob_lower, ev_lower

In [259]:
from itertools import permutations
def getcnt(c):
    return set(permutations((1 if i < c else 0 for i in range(N))))

In [317]:
lk = {} # tup (topm) + (botm) to totalProb, cEV
PP = {}
EE = {}
# Get base
p, ev = simStart()
for i in range(N):
    tt = tuple(1 if q != i else 0 for q in range(5)) + (1, 1, 1, 1, 1)
    PP[tt] = [0 for _ in range(N)]
    EE[tt] = [0 for _ in range(N)]
    PP[tt][i] = p[i]
    EE[tt][i] = ev[i]

In [318]:
uc, lc = 4, 5
while uc or lc:
    print(uc, lc)
    ou = getcnt(uc)
    nl = getcnt(lc)
    tc = []
    for e in ou:
        for e2 in nl:
            tc.append((e, e2))
    if lc > uc:
        # Go through and such
        for umask, lmask in tc:
            probs = PP[tuple(umask + lmask)]
            for i in range(5):
                if not umask[i]:
                    nprob, nev = simTop(i, lmask)
                    for q in range(5):
                        if lmask[q]:
                            ulmask = tuple(lmask[j] if j != q else 0 for j in range(N))
                            # Prob of going from i to q
                            if umask + ulmask not in PP:
                                PP[umask + ulmask] = [0 for _ in range(N)]
                                EE[umask + ulmask] = [0 for _ in range(N)]
                            PP[umask + ulmask][q] += probs[i] * nprob[q]
                            # EE[umask + ulmask][q] += probs[i] * nev[q]
                            # probs[i] * nprob[q]
        lc -= 1
    else:
        # Take from upper
        # Go through and such
        for umask, lmask in tc:
            probs = PP[tuple(umask + lmask)]
            for i in range(5):
                if not lmask[i]:
                    nprob, nev = simBot(i, umask)
                    for q in range(5):
                        if umask[q]:
                            uumask = tuple(umask[j] if j != q else 0 for j in range(N))
                            # Prob of going from i to q
                            if uumask + lmask not in PP:
                                PP[uumask + lmask] = [0 for _ in range(N)]
                                EE[uumask + lmask] = [0 for _ in range(N)]
                            PP[uumask + lmask][q] += probs[i] * nprob[q]
                            # EE[uumask + lmask][q] += probs[i] * nev[q]
                            # probs[i] * nprob[q]
        uc -= 1

4 5
4 4
3 4
3 3
2 3
2 2
1 2
1 1
0 1


In [319]:
uc, lc = 4, 5
while uc or lc:
    print(uc, lc)
    ou = getcnt(uc)
    nl = getcnt(lc)
    tc = []
    for e in ou:
        for e2 in nl:
            tc.append((e, e2))
    if lc > uc:
        # Go through and such
        for umask, lmask in tc:
            probs = PP[tuple(umask + lmask)]
            evs = EE[tuple(umask + lmask)]
            for i in range(5):
                if not umask[i]:
                    nprob, nev = simTop(i, lmask)
                    for q in range(5):
                        if lmask[q]:
                            ulmask = tuple(lmask[j] if j != q else 0 for j in range(N))
                            # Prob of going from i to q
                            if umask + ulmask not in PP:
                                PP[umask + ulmask] = [0 for _ in range(N)]
                                EE[umask + ulmask] = [0 for _ in range(N)]
                            if PP[umask + ulmask][q]:
                                cev = nev[q] + evs[i]
                                EE[umask + ulmask][q] += (probs[i] * nev[q]) + nprob[q] * evs[i]
                            # probs[i] * nprob[q]
        lc -= 1
    else:
        # Take from upper
        # Go through and such
        for umask, lmask in tc:
            probs = PP[tuple(umask + lmask)]
            evs = EE[tuple(umask + lmask)]
            for i in range(5):
                if not lmask[i]:
                    nprob, nev = simBot(i, umask)
                    for q in range(5):
                        if umask[q]:
                            uumask = tuple(umask[j] if j != q else 0 for j in range(N))
                            # Prob of going from i to q
                            if uumask + lmask not in PP:
                                PP[uumask + lmask] = [0 for _ in range(N)]
                                EE[uumask + lmask] = [0 for _ in range(N)]
                            if PP[uumask + lmask][q]:
                                EE[uumask + lmask][q] += (probs[i] * nev[q]) + nprob[q] * evs[i]
                            # probs[i] * nprob[q]
        uc -= 1

4 5
4 4
3 4
3 3
2 3
2 2
1 2
1 1
0 1


In [323]:
E = 0
for i in range(N):
    A = EE[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)][i] * (1 / (PP[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)][i]))
    E += A
E / 5

426.0117061321183

In [327]:
sum(EE[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)])

430.08824671668674

In [310]:
EE[(1, 1, 0, 1, 1, 1, 1, 1, 1, 1)], EE[(1, 1, 0, 1, 1, 1, 1, 0, 1, 1)]

([0, 0, 4.916697347118621, 0, 0], [0, 0, 2.9523153748504947, 0, 0])

In [311]:
PP[(1, 1, 0, 1, 1, 1, 1, 1, 1, 1)], PP[(1, 1, 0, 1, 1, 1, 1, 0, 1, 1)]

([0, 0, 0.25657853874660935, 0, 0], [0, 0, 0.05301982143702841, 0, 0])

In [312]:
EE[(1, 1, 0, 1, 1, 1, 1, 1, 1, 1)][2] * (1 / PP[(1, 1, 0, 1, 1, 1, 1, 1, 1, 1)][2]),\
EE[(1, 1, 0, 1, 1, 1, 1, 0, 1, 1)][2] * (1 / PP[(1, 1, 0, 1, 1, 1, 1, 0, 1, 1)][2])

(19.16254325531969, 55.68323873661017)

In [80]:
# [5, 5 -> 4, (5)] -> 4, 4

In [316]:
from random import randint
def sim():
    lk = [
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
    ]
    x = 2
    y = 2
    c = 0
    hasSeed = False
    for M in range(200_000):
        if sum(lk[0]) == 1 and sum(lk[4]) == 4:
            if lk[0][2] == 1 and lk[4][2] == 0:
                return M
            return -1
        y, x = adj[y][x][randint(0, len(adj[y][x]) - 1)]
        if hasSeed and y == 0 and lk[y][x] == 0:
            lk[y][x] = 1
            hasSeed = False
        if not hasSeed and y == 4 and lk[y][x] == 1:
            lk[y][x] = 0
            hasSeed = True
    return M
data = tuple(e for e in [sim() for _ in range(100_000)] if e != -1)
sum(data) / len(data)

51.148684712161646

In [326]:
from random import randint
def sim():
    lk = [
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
    ]
    x = 2
    y = 2
    c = 0
    hasSeed = False
    for M in range(100_000):
        if sum(lk[0]) == 5:
            return M
        y, x = adj[y][x][randint(0, len(adj[y][x]) - 1)]
        if hasSeed and y == 0 and lk[y][x] == 0:
            lk[y][x] = 1
            hasSeed = False
        if not hasSeed and y == 4 and lk[y][x] == 1:
            lk[y][x] = 0
            hasSeed = True
sum(sim() for _ in range(10_000)) / 10_000

429.6127

In [197]:
from random import randint
def verify():
    TT = [0 for _ in range(5)]
    SS = [0 for _ in range(5)]
    for _ in range(100_000):
        lk = [
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [1, 1, 1, 1, 1],
        ]
        x = 3
        y = 0
        c = 0
        hasSeed = False
        for M in range(100_000):
            y, x = adj[y][x][randint(0, len(adj[y][x]) - 1)]
            if not hasSeed and y == 4 and lk[y][x] == 1:
                TT[x] += 1
                SS[x] += (M + 1)
                break
    return TT, SS